In [ ]:
import copy
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource
from pychunkedgraph.graph.utils.inspect import dfs_print_node

In [ ]:
ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

# edges = np.array([[94803535283198313, 94803466563722624]],dtype=np.uint64)
# between 8
edges = np.array([[94524946524577880, 94595315268752176]],dtype=np.uint64)

print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_parents(edges[0]))
roots_old = cg.get_roots(edges[0])
print(roots_old)

In [ ]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    done, new_ids = add_edges(cg, atomic_edges=edges)    
print()
print(new_ids[12])
roots = new_ids[12]

In [ ]:
for i in range(2, 12):
    print(i, new_ids[i])

In [ ]:
# dfs_print_node(cg, 595601050722147411, limit=2, sep="    ")
cg.get_all_parents_dict(600104650348842218)

In [ ]:
dfs_print_node(cg, 864691128455135396, limit=2, sep="    ")

In [ ]:
limit = 1
leaves = []
for root in [*roots_old, roots[1]]:
    print(root)
    leaves.append([])
    children = np.array([root], dtype=np.uint64)
    mask = cg.get_chunk_layers(children) > limit
    while np.any(mask):
        children = cg.get_children(children[mask], flatten=True)
        mask = cg.get_chunk_layers(children) > limit
        leaves[-1].append(children[~mask])
    leaves[-1] = np.concatenate(leaves[-1])
    print(len(leaves[-1]))
    
print()    
print([len(x) for x in leaves])
    
old = np.unique(np.concatenate([leaves[0], leaves[1]]))
new = np.unique(leaves[2])
print(len(old), len(new))

In [ ]:
cg.get_children(np.array([238589352450461190, 238589627328365341, 238730089938813204,
       238729815060909117, 455162629527435310, 382964332360706811],
      dtype=np.uint64), flatten=True)

In [ ]:
print(len(np.setdiff1d(old, new)))
diff = np.setdiff1d(old, new)
roots = cg.get_roots(diff)
print(np.unique(roots, return_counts=True))

In [ ]:
cg.get_parent(600104650349470359)